**深度卷积网络一开始面临的最主要的问题是梯度消失和梯度爆炸。**\
那什么是梯度消失和梯度爆炸呢？
所谓**梯度消失**，就是在深层神经网络的训练过程中，计算得到的梯度越来越小，使得权值得不到更新的情形，这样算法也就失效了。\
而**梯度爆炸**是指在神经网络训练过程中梯度变得越来越大，权值得到疯狂更新的情形，这样算法得不到收敛，模型也就失效了。当然，其间通过设置 relu 和归一化激活函数层等手段使得我们很好的解决这些问题。\
但当我们将网络层数加到更深时却发现训练的准确率在逐渐降低。这种并不是由过拟合造成的神经网络训练数据识别准确率降低的现象我们称之为**退化**（degradation）。其原因可能是：`随着网络的加深，一些层通常是没有必要出现的，如果训练好的参数随着后面的网络扰动，会被类似于白噪音的问题使参数重新偏移变差。`

![在这里插入图片描述](https://img-blog.csdnimg.cn/20200605204500590.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)

> 由上图我们可以看到 56 层的普通卷积网络不管是在训练集还是测试集上的训练误差都要高于 20 层的卷积网络。是个典型的退化现象。
> 
> 这退化问题不解决，咱们的深度学习就无法 go deeper. 于是何凯明等一干大佬就发明了残差网络 ResNet.

>  何凯明给出的创新在于给网络之间添加一个捷径（shortcuts）或者也叫跳跃连接（skip connection），这使得捷径之间之间的网络能够学习一个恒等函数，使得在加深网络的情形下训练效果至少不会变差。
> 
> 残差块的基本结构如下：

![在这里插入图片描述](https://img-blog.csdnimg.cn/20200605204714659.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)

> 你可能会问凭什么加了一条从输入到输出的捷径网络就能防止退化训练更深层的卷积网络？或是是说残差网络为什么能有效？我们将上述残差块的两层输入输出符号改为 和 ，相应的就有：
$$a^{[l+2]}=g(W^{[l+2]}a^{[l+1]}+b^{[l+2]}+a^{[l]}) (g=relu)\\
在网络中加入 L2 正则化进行权值衰减或者其他情形下，l+2 层的权值 W 是很容易衰减为零的，假设b=0就有 \\
恒等式:a^{[l+2]}=a^{[l]}+min$$
![在这里插入图片描述](https://img-blog.csdnimg.cn/20200605230303544.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)
> 看我画的红圈圈，如果是**plain网络，是没有1的，那么梯度是非常小的，很多非常小的梯度链式相乘变得更小，梯度消失**。因为有了**short cut加上了1，梯度变成比1大一点点(梯度为1+min_num)，在这个环节上保持了链式梯度的稳定性。**
就是说加上short cut的前后，在网络退化问题上，由输入输出较大的线性变化，变成了基本相当的恒等映射。
如果加上short cut之后变成完全恒等的映射，梯度变成1，那么不加任何模块也能达到这个效果。
实际上，**虽然F(x)->0，但F(x)并不等于0，**,用吴恩达的话说：\
**当然我们的目标不仅仅是保持网络效率，还要提升它的效率。想象一下，如果这些（残差模块中的）隐层单元学到一些有用信息，那么它可能比学习恒等函数表现得更好。
加上残差模块，能确定网络性能不会受到（较大）影响（因F(x)->0），很多时候甚至可以提高效率（F(x)虽然很小，还是学习到了新的特征），或者说至少不会降低网络效率。因此创建类似残差网络可以提升性能。**
>> **Plain+残差模块**：梯度为1+min_num，基本恒等映射，但就是（基本恒等映射-完全恒等映射）的F(x)可能学习到了有用的信息，使得网络进一步优化。\
>> **不加F(x)，只加X**：梯度为1，完全恒等映射，对网络优化没有贡献。\
>> **不加X的平凡网络**：梯度为min_num，加强了梯度消失；映射的线性变化很大，过大，可能把已经优化好的参数又改变了，是网络退化的原因。\
>> **把一个深度网络中的层都搞成残差模块**：梯度都变成了1+min_num，链式乘起来会发生梯度爆炸的事件，会导致训练不收敛，因此必须中间有几个min_num，以保证最初几层的梯度参数在一个合理范围之内。
> 由很多个残差块组成的残差网络如下图右图所示：
![在这里插入图片描述](https://img-blog.csdnimg.cn/20200605205317804.png)

## 残差网络 resnet50 的 keras 实现

要实现一个残差块，关键在于实现一个跳跃连接。实际处理中跳跃连接会随着残差块输入输出大小的不同而分为两种。一种是输入输出一致情况下的 **第一种残差块**`Identity Block`，另一种则是输入输出不一致情形下的 **第二种残差块**`Convolutional Block`，顾名思义，就是跳跃连接中包含卷积操作，用来使得输入输出一致。且看二者的 keras 实现方法。

In [3]:
from keras.layers import Conv2D, BatchNormalization,\
    Activation, Add, Input, ZeroPadding2D, MaxPool2D, AveragePooling2D, Flatten, Dense, Lambda
from keras.initializers import glorot_uniform
from keras.models import Model
from keras.utils import plot_model
import numpy as np
from keras.datasets import cifar10
# from keras.utils import to_categorical
from tensorflow.examples.tutorials.mnist import input_data

## 恒等块

![在这里插入图片描述](https://img-blog.csdnimg.cn/20200605205655575.jpg?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)

> kernel_size、stride都为(1,1)这样深度大小不变\
> 即使kernel_size=(f,f)，padding='same'也保证深度大小与输入相同\
> BatchNormalization:批量归一化

In [4]:
def identity_block(X, f, filters, stage, block):
    conv_name = f"res{stage}{block}_branch"
    bn_name = f"bn{stage}{block}_branch"
    f1, f2, f3 = filters
    X_shortcut = X
#     第一个组件
    X = Conv2D(filters=f1, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name=conv_name+"2a", kernel_initializer=glorot_uniform(0))(X)
    X = BatchNormalization(axis=3, name=bn_name+"2a")(X)
    X = Activation('relu')(X)
#     第二个组件
    X = Conv2D(f2, kernel_size=(f, f), strides=(1, 1), padding='same',
               name=conv_name+"2b", kernel_initializer=glorot_uniform(0))(X)
    X = BatchNormalization(axis=3, name=bn_name+"2b")(X)
    X = Activation('relu')(X)
#     第三个组件
    X = Conv2D(f3, kernel_size=(1, 1), strides=(1, 1), padding='valid',
               name=conv_name+"2c", kernel_initializer=glorot_uniform(0))(X)
    X = BatchNormalization(axis=3, name=bn_name+"2c")(X)
#     关键一步
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

## 卷积块

![在这里插入图片描述](https://img-blog.csdnimg.cn/20200605205744580.png)

In [5]:
def convolutional_block(X, f, filters, stage, block, s=2):
    conv_name = f"res{stage}{block}_branch"
    bn_name = f"bn{stage}{block}_branch"
    f1, f2, f3 = filters
    X_shortcut = X
#     第一个组件
    X = Conv2D(f1, (1, 1), strides=(s, s), name=conv_name+"2a",
               padding='valid', kernel_initializer=glorot_uniform(0))(X)
    X = BatchNormalization(axis=3, name=bn_name+"2a")(X)
    X = Activation('relu')(X)
#     第二个组件
    X = Conv2D(f2, (f, f), strides=(1, 1), name=conv_name+"2b",
               padding='same', kernel_initializer=glorot_uniform(0))(X)
    X = BatchNormalization(axis=3, name=bn_name+"2b")(X)
    X = Activation('relu')(X)
#     第三个组件
    X = Conv2D(f3, (1, 1), strides=(1, 1), name=conv_name+"2c",
               padding='valid', kernel_initializer=glorot_uniform(0))(X)
    X = BatchNormalization(axis=3, name=bn_name+"2c")(X)
#     改造 X_shortcut
    X_shortcut = Conv2D(f3, (1, 1), strides=(
        s, s), name=conv_name+"1", kernel_initializer=glorot_uniform(0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name+"1")(X_shortcut)
#     关键一步
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    return X

## 残差网络 resnet50 的 keras 实现

resnet50共有四组block,每组分别是 3 4 6 3个block,每个block里面有三个卷积层
另外最开始都有一个单独的卷积层
> （3+4+6+3）*3+1=49 层卷积\
49+最后1层全连接

所以共有50层网络（有待学习的参数，pooling层不需要参数）
![在这里插入图片描述](https://img-blog.csdnimg.cn/20200605215959924.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L0lfYW1fYV9idWdlcg==,size_16,color_FFFFFF,t_70)

![在这里插入图片描述](https://img-blog.csdnimg.cn/20200605205823798.png)

In [6]:
def ResNet50(input_shape=(64, 64, 3), classes=6):
    X_input = Input(input_shape)
    X = ZeroPadding2D((3, 3))(X_input)
#     stage:1
    X = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2),
               name="conv", kernel_initializer=glorot_uniform(0))(X)
    X = BatchNormalization(axis=3, name="bn")(X)
    X = Activation('relu')(X)
    X = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(X)
#     stage:2
    X = convolutional_block(
        X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, f=3, filters=[64, 64, 256], stage=2, block='b')
    X = identity_block(X, f=3, filters=[64, 64, 256], stage=2, block='c')
#     stage:3
    X = convolutional_block(
        X, f=3, filters=[128, 128, 512], stage=3, block='a', s=1)
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block='b')
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block='c')
    X = identity_block(X, f=3, filters=[128, 128, 512], stage=3, block='d')
#     stage:4
    X = convolutional_block(
        X, f=3, filters=[256, 256, 1024], stage=4, block='a', s=2)
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='b')
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='c')
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='d')
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='e')
    X = identity_block(X, f=3, filters=[256, 256, 1024], stage=4, block='f')
#     stage:5
    X = convolutional_block(
        X, f=3, filters=[512, 512, 2048], stage=5, block='a', s=2)
    X = identity_block(X, f=3, filters=[512, 512, 2048], stage=5, block='b')
    X = identity_block(X, f=3, filters=[512, 512, 2048], stage=5, block='c')
#     AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2), padding='same')(X)
    X = Flatten()(X)
    X = Dense(classes, activation='softmax',
              name=f'fc{classes}', kernel_initializer=glorot_uniform(0))(X)
#     Model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')
    return model

In [8]:
# X_train,y_train,X_test,y_test=cifar10.load_data()

# X_test=to_categorical(X_test,10)
# y_test=to_categorical(y_test,10)
mnist = input_data.read_data_sets('fashion-mnist/', one_hot=True)
X_train, X_test = mnist.train.images, mnist.test.images
X_train = X_train.reshape(len(X_train), 28, 28, -1)
X_test = X_test.reshape(len(X_test), 28, 28, -1)
y_train = mnist.train.labels
y_test = mnist.test.labels
input_shape = X_train.shape[1:]
classes = 10
model = ResNet50(input_shape, classes)
print(f"***: {model.output}")
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=12, batch_size=100)

Extracting fashion-mnist/train-images-idx3-ubyte.gz
Extracting fashion-mnist/train-labels-idx1-ubyte.gz
Extracting fashion-mnist/t10k-images-idx3-ubyte.gz
Extracting fashion-mnist/t10k-labels-idx1-ubyte.gz
***: Tensor("fc10_1/Softmax:0", shape=(?, 10), dtype=float32)


KeyboardInterrupt: 

In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'D:\software\Graphviz2.38\bin'
plot_model(model, to_file='multilayer_perceptron_graph.png',
           show_shapes=True, show_layer_names=True)